# RQ1: holding N and mean degree fixed, how does network architecture affect long-run cooperation and local assortment?

## Imports and Config

In [9]:
from Prison import experiment
from Prison import payoff_matrices
from Prison import Agent, Network, NetworkSimulation
from Prison import ImitationStrategy, ReinforcementLearningStrategy, TitForTatStrategy

import math
import os
import random
import warnings
import inspect
import joblib

import numpy as np
import pandas as pd
import networkx as nx
from pathlib import Path
from functools import partial
import matplotlib.pyplot as plt
from IPython.display import Image

from dataclasses import dataclass
from joblib import Parallel, delayed
from contextlib import contextmanager

import matplotlib.patheffects as pe
import matplotlib.patches as mpatches
from matplotlib.colors import ListedColormap
from matplotlib.ticker import PercentFormatter
from matplotlib.animation import FuncAnimation, PillowWriter
from tqdm.auto import tqdm

PLOT_DIR = Path("images")
PLOT_DIR.mkdir(parents=True, exist_ok=True)

warnings.filterwarnings("ignore", category=FutureWarning)

In [10]:
RUN_ARCH = False
RUN_PLOTS = True
RUN_STRUCTURE = True
RUN_ROBUSTNESS = True
RUN_ANIMATIONS = False
RUN_UNCERTAINTY = True
RUN_PARAM_SWEEPS = True

RUN_ANIMATIONS_PARALLEL = True
RUN_PARALLEL = True
PARALLEL_JOBS = 7
USE_TQDM = True
ANIMATION_JOBS = None

SEEDS = list(range(30))
N = 20 * 20
TAIL = 25
MEAN_K = 4
STEPS = 100
METRIC_STRIDE = 1

In [11]:
def _safe_plot_name(name):
    """Return a filesystem-safe plot name."""
    return "".join(ch if ch.isalnum() or ch in "-_." else "_" for ch in name).strip(
        "._"
    )


def save_fig(fig, name, ext="png", dpi=200):
    """Save and close a matplotlib figure, returning the path."""
    safe = _safe_plot_name(name) or "plot"
    path = PLOT_DIR / f"{safe}.{ext}"
    path.parent.mkdir(parents=True, exist_ok=True)
    fig.savefig(path, dpi=dpi, bbox_inches="tight")
    plt.close(fig)
    return path

## Helpers

In [12]:
def _empty_network_stats():
    """Return defaults for empty graph metrics."""
    return {
        "avg_clustering": 0.0,
        "transitivity": 0.0,
        "avg_shortest_path": np.nan,
        "approx_diameter": np.nan,
        "degree_cv": np.nan,
        "degree_assortativity": np.nan,
        "mean_degree": np.nan,
    }


def _degree_stats(graph):
    """Return mean degree and coefficient of variation."""
    degrees = np.array([d for _, d in graph.degree()], dtype=float)
    degree_mean = float(degrees.mean()) if len(degrees) else 0.0
    degree_std = float(degrees.std()) if len(degrees) else 0.0
    degree_cv = degree_std / degree_mean if degree_mean > 0 else np.nan
    return degree_mean, degree_cv


def _path_metrics(graph, include_path_metrics):
    """Return average path length and approximate diameter."""
    avg_path = np.nan
    approx_diam = np.nan
    if include_path_metrics and graph.number_of_edges() > 0:
        # Path-based stats on the giant component only.
        gc_nodes = max(nx.connected_components(graph), key=len)
        gc = graph.subgraph(gc_nodes)
        if gc.number_of_nodes() > 1:
            avg_path = nx.average_shortest_path_length(gc)
            approx_diam = nx.approximation.diameter(gc)
    return avg_path, approx_diam


def _graph_metrics(graph):
    """Return clustering, transitivity, and assortativity."""
    return (
        float(nx.approximation.average_clustering(graph)),
        float(nx.transitivity(graph)),
        float(nx.degree_assortativity_coefficient(graph)),
    )


def network_statistics(graph, include_path_metrics=True):
    """Compute graph-level controls/mediators."""
    if graph.number_of_nodes() == 0:
        return _empty_network_stats()

    degree_mean, degree_cv = _degree_stats(graph)
    avg_path, approx_diam = _path_metrics(graph, include_path_metrics)
    avg_clustering, transitivity, assortativity = _graph_metrics(graph)

    return {
        "avg_clustering": float(avg_clustering),
        "transitivity": float(transitivity),
        "avg_shortest_path": float(avg_path),
        "approx_diameter": float(approx_diam),
        "degree_cv": float(degree_cv),
        "degree_assortativity": float(assortativity),
        "mean_degree": float(degree_mean),
    }


def _mean_or_nan(values):
    """Return the mean of a list or NaN when empty."""
    return float(np.mean(values)) if values else np.nan


def _tail_start(steps, window):
    """Return the index where the tail window starts."""
    return max(0, steps - window)


def _metric_stride(value):
    """Return a sanitized metric stride value."""
    return max(1, int(value))


def _should_sample(t, tail_start, metric_stride):
    """Return True when the step should be sampled."""
    return t >= tail_start and (t - tail_start) % metric_stride == 0


def _append_tail_metrics(model, coop_vals, assort_vals, largest_vals):
    """Append tail metrics for the current model state."""
    state01 = model.state01_array()
    coop_vals.append(float((state01 == 0).mean()))
    assort_vals.append(model.cooperation_assortment(state01)["assortment_r"])
    cmet = model.cooperation_metrics(state01)
    largest_vals.append(cmet["largest_coop_cluster"])


def trailing_window_metrics(model, steps, window=100, metric_stride=1):
    """Average cooperation/assortment over the last window of steps."""
    coop_vals = []
    assort_vals = []
    largest_vals = []

    metric_stride = _metric_stride(metric_stride)
    tail_start = _tail_start(steps, window)

    for t in range(steps):
        model.step()
        if not _should_sample(t, tail_start, metric_stride):
            continue
        _append_tail_metrics(model, coop_vals, assort_vals, largest_vals)

    return {
        "tail_mean_coop_frac": _mean_or_nan(coop_vals),
        "tail_mean_assortment": _mean_or_nan(assort_vals),
        "tail_mean_largest_cluster": _mean_or_nan(largest_vals),
    }


def run_architecture_sweep(
    configs,
    n,
    payoff_matrix,
    strategy_class,
    strategy_kwargs,
    seeds,
    max_steps=400,
    tail_window=120,
    include_path_metrics=False,
    metric_stride=1,
    use_tqdm=True,
    strategy_label=None,
    matrix_label=None,
):
    """Run the same game across multiple network architectures."""
    rows = []
    strategy_kwargs = strategy_kwargs or {}

    tasks = [(cfg, seed) for cfg in configs for seed in seeds]
    iterator = tasks
    if use_tqdm:
        iterator = tqdm(tasks, desc="Architecture sweep", total=len(tasks))

    for cfg, seed in iterator:
        kind = cfg["kind"]
        graph_kwargs = cfg.get("graph_kwargs", {})
        label = cfg.get("label", kind)
        mean_k_cfg = cfg.get("mean_k")
        param_name = cfg.get("param_name")
        param_value = cfg.get("param_value")

        model = NetworkSimulation(
            kind=kind,
            n=n,
            seed=seed,
            rounds=max_steps,
            payoff_matrix=payoff_matrix,
            strategy=strategy_class,
            strategy_kwargs=strategy_kwargs,
            store_snapshots=False,
            **graph_kwargs,
        )

        row = {
            "label": label,
            "kind": kind,
            "seed": seed,
            "n": model.graph.number_of_nodes(),
            "mean_k": mean_k_cfg,
            "param_name": param_name,
            "param_value": param_value,
        }
        if strategy_label is not None:
            row["strategy"] = strategy_label
        if matrix_label is not None:
            row["matrix"] = matrix_label

        for k, v in graph_kwargs.items():
            if isinstance(v, (int, float, str)):
                row[k] = v

        row.update(network_statistics(model.graph, include_path_metrics))
        row.update(
            trailing_window_metrics(
                model,
                steps=max_steps,
                window=tail_window,
                metric_stride=metric_stride,
            )
        )
        rows.append(row)

    return pd.DataFrame(rows)


def _freeze_value(value):
    """Return a hashable representation of nested values."""
    if isinstance(value, dict):
        return tuple(sorted((k, _freeze_value(v)) for k, v in value.items()))
    if isinstance(value, (list, tuple)):
        return tuple(_freeze_value(v) for v in value)
    return value


def _job_key(cfg, strategy_label, matrix_label, strategy_kwargs):
    """Return a stable key for a sweep job."""
    return (
        cfg.get("label", cfg.get("kind")),
        cfg.get("kind"),
        _freeze_value(cfg.get("graph_kwargs", {})),
        cfg.get("mean_k"),
        cfg.get("param_name"),
        cfg.get("param_value"),
        strategy_label,
        _freeze_value(strategy_kwargs or {}),
        matrix_label,
    )


def add_jobs(
    job_index, configs, strategy_configs, matrix_configs, include_path_metrics
):
    """Add jobs to a shared index and return the job keys in order."""
    keys = []
    for strat_label, strat_cls, strat_kwargs in strategy_configs:
        for mat_label, payoff in matrix_configs:
            for cfg in configs:
                key = _job_key(cfg, strat_label, mat_label, strat_kwargs)
                keys.append(key)
                job = job_index.get(key)
                if job is None:
                    job = {
                        "cfg": cfg,
                        "strategy_label": strat_label,
                        "strategy_class": strat_cls,
                        "strategy_kwargs": strat_kwargs or {},
                        "matrix_label": mat_label,
                        "payoff_matrix": payoff,
                        "include_path_metrics": bool(include_path_metrics),
                    }
                    job_index[key] = job
                else:
                    job["include_path_metrics"] = (
                        job["include_path_metrics"] or include_path_metrics
                    )
    return keys


def _graph_key(kind, graph_kwargs, n, seed):
    """Return a stable key for cached graphs."""
    return (kind, _freeze_value(graph_kwargs), int(n), int(seed))


def _supports_graph_arg():
    """Return True if NetworkSimulation accepts a graph kwarg."""
    try:
        return "graph" in inspect.signature(NetworkSimulation.__init__).parameters
    except (TypeError, ValueError):
        return False


def _default_n_jobs(requested):
    """Return a safe default for parallel workers."""
    if requested is not None:
        return int(requested)
    cpu = os.cpu_count() or 1
    return max(1, cpu - 1)


def _simulation_tasks(job_index, seeds):
    """Yield (job_key, job, seed) tuples for the batch."""
    for key, job in job_index.items():
        for seed in seeds:
            yield key, job, seed


@contextmanager
def tqdm_joblib(tqdm_object):
    """Context manager to patch joblib to report into tqdm."""

    class TqdmBatchCompletionCallback(joblib.parallel.BatchCompletionCallBack):
        def __call__(self, *args, **kwargs):
            tqdm_object.update(n=self.batch_size)
            return super().__call__(*args, **kwargs)

    old_callback = joblib.parallel.BatchCompletionCallBack
    joblib.parallel.BatchCompletionCallBack = TqdmBatchCompletionCallback
    try:
        yield tqdm_object
    finally:
        joblib.parallel.BatchCompletionCallBack = old_callback
        tqdm_object.close()


def _run_single_job(job_key, job, seed, n, max_steps, tail_window, metric_stride):
    """Run a single (job, seed) simulation and return its row."""
    cfg = job["cfg"]
    kind = cfg["kind"]
    graph_kwargs = cfg.get("graph_kwargs", {})
    label = cfg.get("label", kind)
    mean_k_cfg = cfg.get("mean_k")
    param_name = cfg.get("param_name")
    param_value = cfg.get("param_value")

    if _supports_graph_arg():
        net = Network()
        net.generate_graph(kind=kind, n=n, seed=seed, **graph_kwargs)
        graph = net.graph
        model = NetworkSimulation(
            kind=kind,
            n=n,
            seed=seed,
            rounds=max_steps,
            payoff_matrix=job["payoff_matrix"],
            strategy=job["strategy_class"],
            strategy_kwargs=job["strategy_kwargs"],
            store_snapshots=False,
            graph=graph,
            **graph_kwargs,
        )
    else:
        model = NetworkSimulation(
            kind=kind,
            n=n,
            seed=seed,
            rounds=max_steps,
            payoff_matrix=job["payoff_matrix"],
            strategy=job["strategy_class"],
            strategy_kwargs=job["strategy_kwargs"],
            store_snapshots=False,
            **graph_kwargs,
        )

    row = {
        "job_key": job_key,
        "label": label,
        "kind": kind,
        "seed": seed,
        "n": model.graph.number_of_nodes(),
        "mean_k": mean_k_cfg,
        "param_name": param_name,
        "param_value": param_value,
        "strategy": job["strategy_label"],
        "matrix": job["matrix_label"],
    }

    for k, v in graph_kwargs.items():
        if isinstance(v, (int, float, str)):
            row[k] = v

    row.update(network_statistics(model.graph, job["include_path_metrics"]))
    row.update(
        trailing_window_metrics(
            model,
            steps=max_steps,
            window=tail_window,
            metric_stride=metric_stride,
        )
    )
    return row


def run_simulation_batch(
    job_index,
    n,
    seeds,
    max_steps=400,
    tail_window=120,
    metric_stride=1,
    parallel=False,
    n_jobs=None,
    use_tqdm=True,
):
    """Run all jobs once and return a DataFrame of results."""
    if not job_index:
        return pd.DataFrame([])

    if not parallel:
        rows = []
        graph_cache = {}
        tasks = list(_simulation_tasks(job_index, seeds))
        iterator = tasks
        if use_tqdm:
            iterator = tqdm(tasks, desc="Simulations", total=len(tasks))

        for key, job, seed in iterator:
            cfg = job["cfg"]
            kind = cfg["kind"]
            graph_kwargs = cfg.get("graph_kwargs", {})
            label = cfg.get("label", kind)
            mean_k_cfg = cfg.get("mean_k")
            param_name = cfg.get("param_name")
            param_value = cfg.get("param_value")

            gkey = _graph_key(kind, graph_kwargs, n, seed)
            graph = graph_cache.get(gkey)
            if graph is None:
                net = Network()
                net.generate_graph(kind=kind, n=n, seed=seed, **graph_kwargs)
                graph = net.graph
                graph_cache[gkey] = graph

            if _supports_graph_arg():
                model = NetworkSimulation(
                    kind=kind,
                    n=n,
                    seed=seed,
                    rounds=max_steps,
                    payoff_matrix=job["payoff_matrix"],
                    strategy=job["strategy_class"],
                    strategy_kwargs=job["strategy_kwargs"],
                    store_snapshots=False,
                    graph=graph,
                    **graph_kwargs,
                )
            else:
                model = NetworkSimulation(
                    kind=kind,
                    n=n,
                    seed=seed,
                    rounds=max_steps,
                    payoff_matrix=job["payoff_matrix"],
                    strategy=job["strategy_class"],
                    strategy_kwargs=job["strategy_kwargs"],
                    store_snapshots=False,
                    **graph_kwargs,
                )

            row = {
                "job_key": key,
                "label": label,
                "kind": kind,
                "seed": seed,
                "n": model.graph.number_of_nodes(),
                "mean_k": mean_k_cfg,
                "param_name": param_name,
                "param_value": param_value,
                "strategy": job["strategy_label"],
                "matrix": job["matrix_label"],
            }

            for k, v in graph_kwargs.items():
                if isinstance(v, (int, float, str)):
                    row[k] = v

            row.update(network_statistics(model.graph, job["include_path_metrics"]))
            row.update(
                trailing_window_metrics(
                    model,
                    steps=max_steps,
                    window=tail_window,
                    metric_stride=metric_stride,
                )
            )
            rows.append(row)

        return pd.DataFrame(rows)

    tasks = list(_simulation_tasks(job_index, seeds))
    if not tasks:
        return pd.DataFrame([])

    n_jobs = _default_n_jobs(n_jobs)
    iterator = (
        delayed(_run_single_job)(
            key, job, seed, n, max_steps, tail_window, metric_stride
        )
        for key, job, seed in tasks
    )

    if use_tqdm:
        with tqdm_joblib(tqdm(total=len(tasks), desc="Simulations")):
            rows = Parallel(n_jobs=n_jobs, prefer="processes")(iterator)
    else:
        rows = Parallel(n_jobs=n_jobs, prefer="processes")(iterator)

    return pd.DataFrame(rows)

## Animations

In [13]:
if RUN_ANIMATIONS:
    matrix_names = ["Snowdrift", "Default"]

    runs = [
        ("Imitation", ImitationStrategy, {}),
        ("Tit-for-Tat", TitForTatStrategy, {}),
        (
            "Reinforcement Learning",
            ReinforcementLearningStrategy,
            {
                "learning_rate": 0.1,
                "epsilon": 0.1,
                "initial_q": 0.0,
            },
        ),
    ]

    graph_setups = [
        ("Grid", "grid", {}, True),
        ("Small-World", "watts_strogatz", {"k": 4, "p": 0.1}, False),
        ("Erdos-Renyi", "erdos_renyi", {"p": 0.1}, False),
    ]

    def _run_animation(task):
        (
            matrix_name,
            graph_label,
            kind,
            graph_kwargs,
            is_grid,
            strat_label,
            strat_cls,
            strat_kwargs,
        ) = task
        matrix = payoff_matrices[matrix_name]
        return experiment(
            network_simulation=NetworkSimulation,
            strategy_class=strat_cls,
            strategy_kwargs=strat_kwargs,
            steps=STEPS,
            seed=42,
            interval=75,
            payoff_matrix=matrix,
            kind=kind,
            n=N,
            is_grid=is_grid,
            title=f"{strat_label} on {matrix_name} ({graph_label})",
            **graph_kwargs,
        )

    animation_tasks = []
    for matrix_name in matrix_names:
        for graph_label, kind, graph_kwargs, is_grid in graph_setups:
            for strat_label, strat_cls, strat_kwargs in runs:
                animation_tasks.append(
                    (
                        matrix_name,
                        graph_label,
                        kind,
                        graph_kwargs,
                        is_grid,
                        strat_label,
                        strat_cls,
                        strat_kwargs,
                    )
                )

    if RUN_ANIMATIONS_PARALLEL:
        n_jobs = _default_n_jobs(ANIMATION_JOBS)
        if USE_TQDM:
            with tqdm_joblib(tqdm(total=len(animation_tasks), desc="Animations")):
                _ = Parallel(n_jobs=n_jobs, prefer="processes")(
                    delayed(_run_animation)(task) for task in animation_tasks
                )
        else:
            _ = Parallel(n_jobs=n_jobs, prefer="processes")(
                delayed(_run_animation)(task) for task in animation_tasks
            )
    else:
        iterator = animation_tasks
        if USE_TQDM:
            iterator = tqdm(animation_tasks, desc="Animations")
        for task in iterator:
            _run_animation(task)

    # display(ani)

## Statistics

In [14]:
STRATEGY_CONFIGS = [
    ("Imitation", ImitationStrategy, {}),
    ("TitForTat", TitForTatStrategy, {}),
    (
        "Reinforcement Learning",
        ReinforcementLearningStrategy,
        {"learning_rate": 0.1, "epsilon": 0.1, "initial_q": 0.0},
    ),
]
MATRIX_CONFIGS = [
    ("Default", payoff_matrices["Default"]),
    ("Snowdrift", payoff_matrices["Snowdrift"]),
]

PLOT_STYLE = {
    "title_size": 13,
    "label_size": 11,
    "tick_size": 10,
    "line_width": 2.0,
    "marker_size": 6,
}

# Global plotting style for readability
plt.style.use("seaborn-v0_8-whitegrid")
plt.rcParams.update(
    {
        "figure.dpi": 120,
        "axes.titleweight": "semibold",
        "axes.titlesize": PLOT_STYLE["title_size"],
        "axes.labelsize": PLOT_STYLE["label_size"],
        "xtick.labelsize": PLOT_STYLE["tick_size"],
        "ytick.labelsize": PLOT_STYLE["tick_size"],
        "legend.frameon": False,
    }
)

PARAM_LABELS = {
    "erdos_renyi": "p (edge prob)",
    "watts_strogatz": "p (rewiring)",
}
NETWORK_LABELS = {
    "erdos_renyi": "Erdos-Renyi",
    "watts_strogatz": "Watts-Strogatz",
}

In [15]:
def _config_cols(df):
    """Return varying config columns present in the dataframe."""
    cols = []
    for col in ["strategy", "matrix"]:
        if col in df.columns and df[col].notna().any() and df[col].nunique() > 1:
            cols.append(col)
    return cols

In [16]:
arch_df = None
param_df = None
robustness_df = None
structure_df = None
base_df = None

arch_keys = []
param_keys = []
robustness_keys = []
structure_keys = []

job_index = {}
ER_PS = None
WS_PS = None
structure_ps = None

if RUN_ARCH:
    arch_configs = [
        {"label": "Grid", "kind": "grid", "graph_kwargs": {}, "mean_k": 4},
        {
            "label": "Erdos-Renyi",
            "kind": "erdos_renyi",
            "graph_kwargs": {"p": MEAN_K / (N - 1)},
            "mean_k": MEAN_K,
        },
        {
            "label": "Watts-Strogatz",
            "kind": "watts_strogatz",
            "graph_kwargs": {"k": MEAN_K, "p": 0.1},
            "mean_k": MEAN_K,
        },
    ]
    arch_keys = add_jobs(
        job_index,
        arch_configs,
        STRATEGY_CONFIGS,
        MATRIX_CONFIGS,
        include_path_metrics=False,
    )

if RUN_PARAM_SWEEPS:
    ER_PS = np.linspace(0.01, 0.08, 6)
    WS_PS = [0.0, 0.02, 0.05, 0.1, 0.2]

    sweep_configs = []
    for p in ER_PS:
        sweep_configs.append(
            {
                "label": "Erdos-Renyi",
                "kind": "erdos_renyi",
                "graph_kwargs": {"p": float(p)},
                "mean_k": float(p * (N - 1)),
                "param_name": "p",
                "param_value": float(p),
            }
        )
    for p in WS_PS:
        sweep_configs.append(
            {
                "label": "Watts-Strogatz",
                "kind": "watts_strogatz",
                "graph_kwargs": {"k": MEAN_K, "p": float(p)},
                "mean_k": MEAN_K,
                "param_name": "p",
                "param_value": float(p),
            }
        )

    param_keys = add_jobs(
        job_index,
        sweep_configs,
        STRATEGY_CONFIGS,
        MATRIX_CONFIGS,
        include_path_metrics=False,
    )

if RUN_ROBUSTNESS:
    robustness_configs = [
        ("Imitation", ImitationStrategy, {}),
        ("Tit-for-Tat", TitForTatStrategy, {}),
        (
            "Reinforcement Learning",
            ReinforcementLearningStrategy,
            {
                "learning_rate": 0.1,
                "epsilon": 0.1,
                "initial_q": 0.0,
            },
        ),
    ]
    robustness_matrices = [
        ("Default", payoff_matrices["Default"]),
        ("Snowdrift", payoff_matrices["Snowdrift"]),
    ]

    robustness_keys = add_jobs(
        job_index,
        [
            {
                "label": "Erdos-Renyi",
                "kind": "erdos_renyi",
                "graph_kwargs": {"p": MEAN_K / (N - 1)},
                "mean_k": MEAN_K,
            },
            {
                "label": "Watts-Strogatz",
                "kind": "watts_strogatz",
                "graph_kwargs": {"k": MEAN_K, "p": 0.1},
                "mean_k": MEAN_K,
            },
        ],
        robustness_configs,
        robustness_matrices,
        include_path_metrics=False,
    )

if RUN_STRUCTURE:
    structure_configs = []
    if RUN_PARAM_SWEEPS and WS_PS is not None:
        structure_ps = np.linspace(float(min(WS_PS)), float(max(WS_PS)), 21)
    else:
        structure_ps = [0.0, 0.05, 0.1]
    for p in structure_ps:
        structure_configs.append(
            {
                "label": "Watts-Strogatz",
                "kind": "watts_strogatz",
                "graph_kwargs": {"k": MEAN_K, "p": float(p)},
                "mean_k": MEAN_K,
                "param_name": "p",
                "param_value": float(p),
            }
        )

    structure_keys = add_jobs(
        job_index,
        structure_configs,
        STRATEGY_CONFIGS,
        MATRIX_CONFIGS,
        include_path_metrics=False,
    )

if job_index:
    base_df = run_simulation_batch(
        job_index,
        n=N,
        seeds=SEEDS,
        max_steps=STEPS,
        tail_window=TAIL,
        metric_stride=METRIC_STRIDE,
        parallel=RUN_PARALLEL,
        n_jobs=PARALLEL_JOBS,
        use_tqdm=USE_TQDM,
    )

if RUN_ARCH and base_df is not None:
    arch_df = base_df[base_df["job_key"].isin(arch_keys)].copy()
    arch_df["tail_mean_largest_cluster_frac"] = (
        arch_df["tail_mean_largest_cluster"] / arch_df["n"]
    )

    arch_group_cols = ["label"] + _config_cols(arch_df)
    arch_summary = (
        arch_df.groupby(arch_group_cols)[
            [
                "tail_mean_coop_frac",
                "tail_mean_assortment",
                "tail_mean_largest_cluster_frac",
                "avg_clustering",
                "mean_degree",
            ]
        ]
        .mean()
        .reset_index()
        .sort_values("tail_mean_coop_frac", ascending=False)
    )

Simulations:   0%|          | 0/5220 [00:00<?, ?it/s]

/Users/tristanfarran/.default-py/lib/python3.13/site-packages/networkx/algorithms/assortativity/correlation.py:302: RuntimeWarning: invalid value encountered in scalar divide
  return float((xy * (M - ab)).sum() / np.sqrt(vara * varb))
/Users/tristanfarran/.default-py/lib/python3.13/site-packages/networkx/algorithms/assortativity/correlation.py:302: RuntimeWarning: invalid value encountered in scalar divide
  return float((xy * (M - ab)).sum() / np.sqrt(vara * varb))
/Users/tristanfarran/.default-py/lib/python3.13/site-packages/networkx/algorithms/assortativity/correlation.py:302: RuntimeWarning: invalid value encountered in scalar divide
  return float((xy * (M - ab)).sum() / np.sqrt(vara * varb))
/Users/tristanfarran/.default-py/lib/python3.13/site-packages/networkx/algorithms/assortativity/correlation.py:302: RuntimeWarning: invalid value encountered in scalar divide
  return float((xy * (M - ab)).sum() / np.sqrt(vara * varb))
/Users/tristanfarran/.default-py/lib/python3.13/site-pac

In [17]:
param_df = None
if RUN_PARAM_SWEEPS and base_df is not None:
    param_df = base_df[base_df["job_key"].isin(param_keys)].copy()
    param_df["tail_mean_largest_cluster_frac"] = (
        param_df["tail_mean_largest_cluster"] / param_df["n"]
    )

    PERCO_THRESH = 0.4
    param_df["percolates"] = param_df["tail_mean_largest_cluster_frac"] >= PERCO_THRESH

    perco_group_cols = ["kind", "param_name", "param_value"] + _config_cols(param_df)
    perco_summary = (
        param_df.groupby(perco_group_cols)["percolates"].mean().reset_index()
    )

In [18]:
robustness_df = None
if RUN_ROBUSTNESS and base_df is not None:
    robustness_df = base_df[base_df["job_key"].isin(robustness_keys)].copy()
    robustness_summary = (
        robustness_df.groupby(["strategy", "matrix", "label"])[["tail_mean_coop_frac"]]
        .mean()
        .sort_values("tail_mean_coop_frac", ascending=False)
    )

In [19]:
structure_df = None
if RUN_STRUCTURE and base_df is not None:
    structure_df = base_df[base_df["job_key"].isin(structure_keys)].copy()
    structure_group_cols = ["param_value"] + _config_cols(structure_df)
    structure_summary = (
        structure_df.groupby(structure_group_cols)[
            ["avg_clustering", "tail_mean_coop_frac"]
        ]
        .mean()
        .reset_index()
        .sort_values("avg_clustering")
    )

    if RUN_PLOTS:
        panel_group_cols = _config_cols(structure_df)
        if panel_group_cols:
            panel_groups = list(structure_df.groupby(panel_group_cols))
        else:
            panel_groups = [(None, structure_df)]

        for group_key, group_df in panel_groups:
            fig, ax = plt.subplots(1, 1, figsize=(6, 4))
            stats = (
                group_df.groupby("param_value")[
                    ["avg_clustering", "tail_mean_coop_frac"]
                ]
                .mean()
                .reset_index()
                .sort_values("avg_clustering")
            )
            x = stats["avg_clustering"].to_numpy()
            y = stats["tail_mean_coop_frac"].to_numpy()
            ax.scatter(
                x,
                y,
                s=(PLOT_STYLE["marker_size"] + 2) ** 2,
                alpha=0.8,
                edgecolors="white",
                linewidth=0.6,
                color="#2F6BFF",
            )
            # Trend line to highlight whether higher clustering aligns with higher cooperation
            if len(x) > 1:
                order = np.argsort(x)
                x_sorted = x[order]
                y_sorted = y[order]
                coeffs = np.polyfit(x_sorted, y_sorted, 1)
                ax.plot(
                    x_sorted,
                    np.polyval(coeffs, x_sorted),
                    color="#FF8C00",
                    linewidth=2,
                    label="Linear trend",
                )
                ax.legend(fontsize=9)

            ax.set_xlabel("Average clustering", fontsize=PLOT_STYLE["label_size"])
            ax.set_ylabel(
                "Cooperation fraction (tail mean)", fontsize=PLOT_STYLE["label_size"]
            )
            title = "Clustering vs cooperation (tail-mean across runs)"
            if panel_group_cols:
                if not isinstance(group_key, tuple):
                    group_key = (group_key,)
                label_bits = [
                    f"{col}={val}" for col, val in zip(panel_group_cols, group_key)
                ]
                title = f"{title} ({' | '.join(label_bits)})"
                plot_suffix = "__".join(label_bits)
            else:
                plot_suffix = "all"
            ax.set_title(title, fontsize=PLOT_STYLE["title_size"])
            ax.grid(True, linestyle=":", alpha=0.4)
            ax.tick_params(labelsize=PLOT_STYLE["tick_size"])
            plt.tight_layout()
            save_fig(fig, f"structure_sensitivity_{plot_suffix}")
            # Binned clustering vs cooperation to show average effect
            if len(stats) >= 4:
                fig2, ax2 = plt.subplots(1, 1, figsize=(6, 4))
                q = min(5, len(stats))
                bins = pd.qcut(stats["avg_clustering"], q=q, duplicates="drop")
                binned = stats.groupby(bins)["tail_mean_coop_frac"].agg(
                    ["mean", "std", "count"]
                )
                x_bin = stats.groupby(bins)["avg_clustering"].mean()
                ax2.errorbar(
                    x_bin,
                    binned["mean"],
                    yerr=binned["std"],
                    marker="o",
                    linewidth=2,
                    color="#2F6BFF",
                )
                ax2.set_xlabel("Average clustering", fontsize=PLOT_STYLE["label_size"])
                ax2.set_ylabel(
                    "Cooperation fraction (tail mean)",
                    fontsize=PLOT_STYLE["label_size"],
                )
                ax2.set_title(
                    "Does higher clustering raise cooperation? (binned means)",
                    fontsize=PLOT_STYLE["title_size"],
                )
                ax2.grid(True, linestyle=":", alpha=0.4)
                ax2.tick_params(labelsize=PLOT_STYLE["tick_size"])
                plt.tight_layout()
                save_fig(fig2, f"clustering_binned_{plot_suffix}")

In [20]:
if RUN_UNCERTAINTY and RUN_PARAM_SWEEPS and param_df is not None:
    uncertainty_group_cols = _config_cols(param_df)
    if uncertainty_group_cols:
        uncertainty_groups = list(param_df.groupby(uncertainty_group_cols))
    else:
        uncertainty_groups = [(None, param_df)]

    for group_key, group_df in uncertainty_groups:
        kinds = ["erdos_renyi", "watts_strogatz"]
        fig, axes = plt.subplots(1, len(kinds), figsize=(12, 3.6), sharey=True)
        if len(kinds) == 1:
            axes = [axes]
        for ax, kind in zip(axes, kinds):
            sub = group_df[group_df["kind"] == kind]
            if sub.empty:
                ax.axis("off")
                continue
            param_name = sub["param_name"].dropna().iloc[0]
            label = NETWORK_LABELS.get(kind, kind)
            stats = (
                sub.groupby("param_value")["tail_mean_coop_frac"]
                .agg(["mean", "std"])
                .sort_index()
            )
            ax.plot(
                stats.index,
                stats["mean"],
                marker="o",
                linewidth=PLOT_STYLE["line_width"],
            )
            ax.fill_between(
                stats.index,
                stats["mean"] - stats["std"],
                stats["mean"] + stats["std"],
                alpha=0.2,
            )
            ax.set_title(
                f"{label}: cooperation (mean ± 1 std)",
                fontsize=PLOT_STYLE["title_size"],
            )
            ax.set_xlabel(
                PARAM_LABELS.get(kind, param_name), fontsize=PLOT_STYLE["label_size"]
            )
            ax.grid(True, linestyle=":", alpha=0.4)
            ax.tick_params(labelsize=PLOT_STYLE["tick_size"])
        axes[0].set_ylabel(
            "Cooperation fraction (tail mean)", fontsize=PLOT_STYLE["label_size"]
        )
        if uncertainty_group_cols:
            if not isinstance(group_key, tuple):
                group_key = (group_key,)
            label_bits = [
                f"{col}={val}" for col, val in zip(uncertainty_group_cols, group_key)
            ]
            fig.suptitle(
                "Seed-to-seed variability in cooperation (mean ± 1 std)"
                + f" ({' | '.join(label_bits)})",
                y=1.02,
                fontsize=12,
            )
            plot_suffix = "__".join(label_bits)
        else:
            plot_suffix = "all"
        plt.tight_layout()
        save_fig(fig, f"uncertainty_{plot_suffix}")

In [21]:
if RUN_PLOTS and RUN_PARAM_SWEEPS and param_df is not None:
    metrics = [
        "tail_mean_coop_frac",
        "tail_mean_assortment",
        "tail_mean_largest_cluster_frac",
    ]
    metric_titles = {
        "tail_mean_coop_frac": "Cooperation (tail mean)",
        "tail_mean_assortment": "Assortment (tail mean)",
        "tail_mean_largest_cluster_frac": "Largest coop cluster (tail mean)",
    }

    panel_group_cols = _config_cols(param_df)
    if panel_group_cols:
        panel_groups = list(param_df.groupby(panel_group_cols))
    else:
        panel_groups = [(None, param_df)]

    for group_key, group_df in panel_groups:
        kinds = list(group_df["kind"].dropna().unique())
        if not kinds:
            continue
        n_rows = len(kinds)
        n_cols = len(metrics)
        fig, axes = plt.subplots(
            n_rows,
            n_cols,
            figsize=(3.6 * n_cols, 2.8 * n_rows),
            sharey="col",
        )
        title = "Network parameter sweeps (tail averages)"
        if panel_group_cols:
            if not isinstance(group_key, tuple):
                group_key = (group_key,)
            label_bits = [
                f"{col}={val}" for col, val in zip(panel_group_cols, group_key)
            ]
            title = f"{title} ({' | '.join(label_bits)})"
            plot_suffix = "__".join(label_bits)
        else:
            plot_suffix = "all"
        fig.suptitle(title, y=1.02, fontsize=12)
        if n_rows == 1:
            axes = [axes]

        for i, kind in enumerate(kinds):
            sub = group_df[group_df["kind"] == kind]
            param_name = sub["param_name"].dropna().iloc[0]
            label = NETWORK_LABELS.get(kind, kind)
            summary = sub.groupby("param_value")[metrics].mean().sort_index()

            row_axes = axes[i] if n_rows > 1 else axes[0]
            for j, metric in enumerate(metrics):
                ax = row_axes[j]
                ax.plot(
                    summary.index,
                    summary[metric],
                    marker="o",
                    linewidth=PLOT_STYLE["line_width"],
                )
                if i == 0:
                    ax.set_title(
                        metric_titles[metric], fontsize=PLOT_STYLE["title_size"]
                    )
                if j == 0:
                    ax.set_ylabel(
                        f"{label}",
                        fontsize=PLOT_STYLE["label_size"],
                    )
                ax.grid(True, linestyle=":", alpha=0.4)
                ax.tick_params(labelsize=PLOT_STYLE["tick_size"])

            mid_col = n_cols // 2
            row_axes[mid_col].set_xlabel(
                PARAM_LABELS.get(kind, param_name),
                fontsize=PLOT_STYLE["label_size"],
            )

        plt.tight_layout()
        save_fig(fig, f"parameter_sweeps_{plot_suffix}")